In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# experiment 1:
# first, creating a set of N randomly generated (n, 1) vectors:
import numpy as np
m = 5
N = 100
vectors = [np.random.rand(*(m, 1)) for _ in range(N)]

c:\Users\Ahsan\anaconda3\envs\messi\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Ahsan\anaconda3\envs\messi\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
c:\Users\Ahsan\anaconda3\envs\messi\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
# finding nearest neighbor of each vector using O(N^2) brute force approach:
nearest_neighbors = {} # the keys will be the indices of the vectors from 0 to N-1, and the corresponding values will be the indices of its nearest neighbor
for i in range(len(vectors)):
    nearest_dist = float('inf')
    nearest_neighbors[i] = None
    for j in range(len(vectors)):
        if j != i:
            dist = np.linalg.norm(vectors[i] - vectors[j])
            if dist < nearest_dist:
                nearest_dist = dist
                nearest_neighbors[i] = j
print (nearest_neighbors) 

{0: 79, 1: 39, 2: 12, 3: 69, 4: 99, 5: 78, 6: 79, 7: 78, 8: 90, 9: 29, 10: 28, 11: 76, 12: 2, 13: 31, 14: 75, 15: 11, 16: 15, 17: 63, 18: 35, 19: 49, 20: 64, 21: 95, 22: 90, 23: 70, 24: 64, 25: 74, 26: 14, 27: 21, 28: 0, 29: 9, 30: 8, 31: 98, 32: 43, 33: 85, 34: 83, 35: 18, 36: 71, 37: 75, 38: 99, 39: 77, 40: 52, 41: 47, 42: 96, 43: 42, 44: 7, 45: 41, 46: 90, 47: 98, 48: 70, 49: 19, 50: 94, 51: 61, 52: 40, 53: 12, 54: 88, 55: 87, 56: 20, 57: 93, 58: 10, 59: 88, 60: 10, 61: 4, 62: 5, 63: 68, 64: 20, 65: 59, 66: 80, 67: 93, 68: 89, 69: 3, 70: 18, 71: 36, 72: 0, 73: 77, 74: 25, 75: 67, 76: 11, 77: 39, 78: 36, 79: 0, 80: 82, 81: 85, 82: 80, 83: 36, 84: 43, 85: 87, 86: 27, 87: 85, 88: 59, 89: 68, 90: 46, 91: 50, 92: 50, 93: 67, 94: 50, 95: 21, 96: 42, 97: 4, 98: 47, 99: 4}


In [4]:
# finding nearest neighbor of each vector using NaiveLSH:
from memristor.engine.model import NaiveLSH
from memristor.crossbar.model import LineResistanceCrossbar
from memristor.devices import StaticMemristor
naive_lsh = NaiveLSH(
    hash_size=3, # adjustable hyperparameter
    crossbar_class=LineResistanceCrossbar,
    crossbar_params={'r_wl': 20, 'r_bl': 20, 'r_in':10, 'r_out':10, 'V_SOURCE_MODE':'|_|'},
    memristor_model_class=StaticMemristor,
    memristor_params={'frequency': 1e8, 'temperature': 273 + 40},
    m=m,
    r=1, # adjustable hyperparameter
)
reps = 5 # adjustable hyperparameter (repetitions of the hashing)
all_bins = []
for _ in range(reps):
    bins = {}
    for i in range(len(vectors)):
        hash = naive_lsh.inference(vectors[i])
        if hash not in bins.keys():
            bins[hash] = [i]
        else:
            bins[hash].append(i)
    for bin in list(bins.values()):
        all_bins.append(bin)
#      {010:[1,5,7], 111:[5,6,7]}

# now at this point all_bins is a list like [[1,2], [1,3,5], ... ] where each element of all_bins is a bin containing indices of vectors that are likely
# to be close to each other. so now to find the nearest neighbor for each vector, we simply iterate through and check only those vectors that share a bin
# with it, so in this case for 1 we would check 2, 3, and 5, to find the nearest neighbor
nearest_neighbors_approx = {}
for i in range(len(vectors)):
    nearest_dist = float('inf')
    nearest_neighbors_approx[i] = None
    for bin in all_bins:
        if i in bin:
            for j in bin:
                if j != i:
                    dist = np.linalg.norm(vectors[i] - vectors[j])
                    if dist < nearest_dist:
                        nearest_dist = dist 
                        nearest_neighbors_approx[i] = j
print (nearest_neighbors_approx)

torch.Size([3, 6])


ValueError: scale < 0